## Thu thập dữ liệu bằng parse HTML - Track_data
- Họ tên: Nguyễn Đức Hiếu
- MSSV: 19120508

## Môi trường code
Dùng phiên bản các package như trong file "min_ds-env.yml"


***
## Import

`Selenium` là một trình duyệt cho phép chúng ta thực hiện các công việc tự động hóa ở trên đó. `Selenium` cho phép điều khiển và code trên trình duyệt. 

In [1]:
import requests
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
s = Service('chromedriver.exe')

Công việc trong phần này là viết hàm `collect_playlist` ở bên dưới. Hàm này dùng để thu thập dữ liệu của playlist trên Soundcloud. Hàm này dùng các input như sau:
 - tracks_urls_file: là chuỗi, cho biết tên của file txt chứa các đường link đến tất cả các playlist, mỗi đường link nằm trên một dòng.
 - tracks_file: là chuỗi , chúng ta sẽ viết danh sách các tracks vào file này. 
 
Chúng ta sẽ sử dụng các biến như sau :
 - urls_file: là danh sách link trong file playlist_urls_file.txt
 - num_of_records : số lượng records
 - url : link dẫn tới các users
 - driver: khởi tạo driver , chúng ta sẽ truyền vào đường dẫn file `chromedriver.exe` . Sẽ cần dowload bản [chromedriver.exe](https://chromedriver.chromium.org/downloads?fbclid=IwAR2J5mdJzt-p7RA28YebsAVB2-hrxhdY2PXfU8kYBKwoBCK-HtH-XDI8T_s) phù hợp với bản chrome mình đang sử dụng.
 - website: link dẫn tới các tracks (url + '/tracks')
 - user[]: tên người đăng tracks
 - title[]: tên của tracks
 - upload_time[]: khoảng thời gian đăng tracks (ago)
 - play_nums[]: số lượt xem của tracks
 - cmt_nums[]: số lượng bình luận của tracks
 - like[]: số lượng lượt thích của tracks
 - repost[]: số lượng phản hồi của tracks


In [2]:
def collect_tracks(tracks_urls_file , tracks_file):
    urls_file = open(tracks_urls_file, "r")
    user = []
    title = []
    upload_time = []
    play_nums = []
    cmt_nums = []
    like = []
    repost = []
    for url in urls_file:
        website = url + '/tracks'
        driver = webdriver.Chrome('chromedriver.exe')
        driver.get(website)
        tracks = driver.find_elements(By.CLASS_NAME,"soundList__item")
        for track in tracks:
            user.append(track.find_element(By.CLASS_NAME,"soundTitle__usernameText").text)
            title.append(track.find_element(By.CLASS_NAME,"soundTitle__title").text)
            upload = track.find_element(By.CLASS_NAME,"relativeTime")
            upload_time.append(upload.find_elements(By.TAG_NAME,"span")[-1].text)
            play_cmt_nums=track.find_element(By.CLASS_NAME,"soundStats")
            play_cmt_text = play_cmt_nums.text
            if(play_cmt_text == ''):
                play_nums.append(0)
                cmt_nums.append(0)
            else:
                play_nums.append(play_cmt_nums.find_element(By.TAG_NAME,'li').get_attribute("title"))
                cmt_nums.append(play_cmt_nums.find_elements(By.TAG_NAME,'li')[-1].get_attribute("title"))
            like.append(track.find_element(By.CLASS_NAME,"sc-button-like").text)
            rep = track.find_element(By.CLASS_NAME,"sc-button-repost").text.isalpha()
            if(rep):
                repost.append(0)
            else:
                repost.append(track.find_element(By.CLASS_NAME,"sc-button-repost").text)
        df = pd.DataFrame({'title': title, 'user':user,'upload_time': upload_time,'number_of_play':play_nums, 'number of comment': cmt_nums, 'like': like, 'repost':repost})
        df.to_csv(tracks_file, index=False,sep="\t")
        if(len(user) >= 1001):
            break
        try:
            soundList__item = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "soundList__item"))
            )
        except:
            driver.quit()
        finally:
            driver.quit()
        

## Tiến hành crawl dữ liệu
Vì thời gian crawl dữ liệu khá lâu (khoảng tầm 30 phút) nên nếu muốn thực thi hàm trên, vui lòng uncomment cell bên dưới để chạy và nên backup file dữ liệu đã crawl

In [3]:
#collect_tracks('url.txt', 'track_data.csv')

## Đọc dữ liệu từ file

In [4]:
courses = pd.read_csv('track_data.csv', sep='\t')
courses

,title,user,upload_time,number_of_play,number of comment,like,repost
0,"GIRIBOY (기리보이) - 내일은 없다 (Feat. lIlBOI, nafla) ...",L2Share♫140,15 days ago,"45,911 plays",23 comments,"1,246",16
1,GIRIBOY (기리보이) - 우린 결국 그렇게 (Feat. Seori) (That...,L2Share♫140,15 days ago,"31,575 plays",13 comments,992,8
2,GIRIBOY (기리보이) - 엉망진창 (What A Mess),L2Share♫140,15 days ago,"58,127 plays",10 comments,"1,948",11
3,GIRIBOY (기리보이) - 그리고 돌아섰다 (You Turned Around),L2Share♫140,15 days ago,"22,788 plays",4 comments,832,10
4,NCT 127 - Sticker,L2Share♫140,1 month ago,"285,815 plays",236 comments,"8,054",86
...,...,...,...,...,...,...,...
1002,"Floating Points, Pharoah Sanders & The London ...",Floating Points,7 months ago,"2,282 plays",5 comments,36,0
1003,"Floating Points, Pharoah Sanders & The London ...",Floating Points,7 months ago,"2,365 plays",10 comments,33,0
1004,"Floating Points, Pharoah Sanders & The London ...",Floating Points,7 months ago,"3,333 plays",12 comments,45,1
1005,"Floating Points, Pharoah Sanders & The London ...",Floating Points,7 months ago,"2,080 plays",10 comments,55,0
